In [8]:
import pandas as pd
cols = ['rate', 'review_text']
dataset = pd.read_csv("/Users/camille/repo/Hetic/projet_gouv/scraping/data_nlp/google_reviews_RGPD.csv", usecols=cols)
dataset['sentiment'] = dataset['rate'].apply(lambda x: 1 if x >= 2.5 else 0)
dataset.dropna(subset=['review_text'], inplace=True)
for_test = dataset.sample(frac=0.2, random_state=200)

In [30]:
cols = ['rate', 'review_text']
dataset = pd.read_csv("/Users/camille/repo/Hetic/projet_gouv/scraping/data_nlp/google_reviews_RGPD.csv", usecols=cols)
dataset['sentiment'] = dataset['rate'].apply(lambda x: 1 if x >= 2.5 else 0)
dataset.dropna(subset=['review_text'], inplace=True)
for_train = dataset.sample(frac=0.8, random_state=200)

In [23]:
dataset.dropna(subset=['review_text'], inplace=True)

In [24]:
dataset['sentiment'] = dataset['rate'].apply(lambda x: 1 if x >= 2.5 else 0)


In [31]:
for_train 
 

,rate,review_text,sentiment
13136,2,Franchement nul,0
5922,5,"Toujours au top, bien reçu, bon conseil et cel...",1
5055,1,Très mauvaise accueil thelephonique a ce jour ...,0
9769,1,C'est une catastrophe ce pole emploi....,0
4277,1,Est-ce normal que le pole emploi ferme a 13h ?,0
...,...,...,...
2398,1,"totalement incompétents, je n'ai jamais été re...",0
14914,1,"On frôle le ridicule!\nIncompétents, je valide...",0
3798,5,J'ai toujours été très bien reçue au Pôle Empl...,1
7208,4,Sa va,1


In [26]:
for_train = dataset.sample(frac=0.8, random_state=200)

In [27]:
for_train  

,rate,review_text,sentiment
13136,2,Franchement nul,0
5922,5,"Toujours au top, bien reçu, bon conseil et cel...",1
5055,1,Très mauvaise accueil thelephonique a ce jour ...,0
9769,1,C'est une catastrophe ce pole emploi....,0
4277,1,Est-ce normal que le pole emploi ferme a 13h ?,0
...,...,...,...
2398,1,"totalement incompétents, je n'ai jamais été re...",0
14914,1,"On frôle le ridicule!\nIncompétents, je valide...",0
3798,5,J'ai toujours été très bien reçue au Pôle Empl...,1
7208,4,Sa va,1


In [31]:
# Chargement du jeu de donnees
from transformers import CamembertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import torch
import seaborn
import pandas as pd
from sklearn import metrics
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertForSequenceClassification, CamembertTokenizer, AdamW


reviews = for_train['review_text'].values.tolist()
sentiments = for_train['sentiment'].values.tolist()
 
# On charge l'objet "tokenizer"de camemBERT qui va servir a encoder
# 'camebert-base' est la version de camembert qu'on choisit d'utiliser
# 'do_lower_case' à True pour qu'on passe tout en miniscule
TOKENIZER =  AutoTokenizer.from_pretrained("camembert-base")
MAX_LENGTH = 128  
# La fonction batch_encode_plus encode un batch de donnees
encoded_batch = TOKENIZER.batch_encode_plus(reviews,
                                            add_special_tokens=True,
                                            max_length=MAX_LENGTH,
                                            padding=True,
                                            truncation=True,
                                            return_attention_mask = True,
                                            return_tensors = 'pt')
 
# On transforme la liste des sentiments en tenseur
sentiments = torch.tensor(sentiments)
 
# On calcule l'indice qui va delimiter nos datasets d'entrainement et de validation
# On utilise 80% du jeu de donnée pour l'entrainement et les 20% restant pour la validation
split_border = int(len(sentiments)*0.8)
 
 
train_dataset = TensorDataset(
    encoded_batch['input_ids'][:split_border],
    encoded_batch['attention_mask'][:split_border],
    sentiments[:split_border])
validation_dataset = TensorDataset(
    encoded_batch['input_ids'][split_border:],
    encoded_batch['attention_mask'][split_border:],
    sentiments[split_border:])
 
 
batch_size = 32
 
# On cree les DataLoaders d'entrainement et de validation
# Le dataloader est juste un objet iterable
# On le configure pour iterer le jeu d'entrainement de façon aleatoire et creer les batchs.
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size)
 
validation_dataloader = DataLoader(
            validation_dataset,
            sampler = SequentialSampler(validation_dataset),
            batch_size = batch_size)

In [32]:
# On la version pre-entrainee de camemBERT 'base'
model = CamembertForSequenceClassification.from_pretrained(
    'camembert-base',
    num_labels = 2)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.weig

In [33]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # Learning Rate
                  eps = 1e-8 # Epsilon
                  )
epochs = 3

/Users/camille/repo/Hetic/projet_gouv/scraping/clean_data_env/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

def preprocess(raw_reviews, sentiments=None):
    encoded_batch = TOKENIZER.batch_encode_plus(raw_reviews,
                                                truncation=True,
                                                pad_to_max_length=True,
                                                return_attention_mask=True,
                                                return_tensors='pt')
    if sentiments:
        sentiments = torch.tensor(sentiments)
        return encoded_batch['input_ids'], encoded_batch['attention_mask'], sentiments
    return encoded_batch['input_ids'], encoded_batch['attention_mask']

def predict(reviews, model):
    with torch.no_grad():
        model.eval()
        input_ids, attention_mask = preprocess(reviews)
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits
        return torch.argmax(logits, dim=1)

def evaluate(reviews, sentiments):
    predictions = predict(reviews, model)
    f1_score = metrics.f1_score(sentiments, predictions, average='weighted', zero_division=0)
    confusion_matrix = metrics.confusion_matrix(sentiments, predictions)
    
    # Plotting the ROC curve
    fpr, tpr, thresholds = metrics.roc_curve(sentiments, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()
    
    # Create a dataframe with predicted and sentiment columns
    df = pd.DataFrame({'Predicted': predictions, 'Sentiment': sentiments})
    return f1_score, confusion_matrix, df


In [29]:
import os
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertForSequenceClassification, CamembertTokenizer, AdamW


class SentimentClassifier:
    def __init__(self):
        self.tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
        self.device = torch.device("mps")
        self.model = None
    
    def preprocess(self, raw_reviews: list, sentiments: list = None) -> tuple:
        encoded_batch = self.tokenizer.batch_encode_plus(
            raw_reviews,
            truncation=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        ).to(self.device)
        
        if sentiments:
            sentiments = torch.tensor(sentiments)
            return encoded_batch['input_ids'], encoded_batch['attention_mask'], sentiments
        
        return encoded_batch['input_ids'], encoded_batch['attention_mask']
    
    def train(self, dataset_file: str):
        # Load the dataset
        cols = ['rate', 'review_text']
        dataset = pd.read_csv(dataset_file, usecols=cols)
        dataset['sentiment'] = dataset['rate'].apply(lambda x: 1 if x >= 2.5 else 0)
        dataset.dropna(subset=['review_text'], inplace=True)
        
        # Sample the dataset
        dataset_sample = dataset.sample(frac=0.8, random_state=200)
        reviews_sample = dataset_sample['review_text'].values.tolist()
        sentiments_sample = dataset_sample['sentiment'].values.tolist()
        
        # Encode the data
        encoded_batch = self.preprocess(reviews_sample, sentiments_sample)
        input_ids, attention_mask, sentiments = encoded_batch
        
        # Split into train and validation datasets
        split_border = int(len(sentiments) * 0.8)
        train_dataset = TensorDataset(
            input_ids[:split_border],
            attention_mask[:split_border],
            sentiments[:split_border]
        )
        validation_dataset = TensorDataset(
            input_ids[split_border:],
            attention_mask[split_border:],
            sentiments[split_border:]
        )
        
        # Set batch size and create data loaders
        batch_size = 10
        train_dataloader = DataLoader(
            train_dataset,
            sampler=RandomSampler(train_dataset),
            batch_size=batch_size
        )
        validation_dataloader = DataLoader(
            validation_dataset,
            sampler=SequentialSampler(validation_dataset),
            batch_size=batch_size
        )
        
        # Create the model
        self.model = CamembertForSequenceClassification.from_pretrained("camembert-base")
        self.model.to(self.device)
        
        # Define the optimizer and loss function
        optimizer = AdamW(self.model.parameters(), lr=1e-5)
        loss_fn = torch.nn.CrossEntropyLoss()
        
        # Training loop
        epochs = 3
        for epoch in range(epochs):
            # Training
            total_train_loss = 0
            self.model.train()
            for step, batch in enumerate(train_dataloader):
                input_ids = batch[0].to(self.device)
                attention_mask = batch[1].to(self.device)
                sentiment = batch[2].to(self.device)
                
                self.model.zero_grad()
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=sentiment
                )
                logits = outputs.logits
                loss = loss_fn(logits.view(-1, self.model.config.num_labels), sentiment.view(-1))
                total_train_loss += loss.item()
                loss.backward()
                optimizer.step()
            
            avg_train_loss = total_train_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  Average training loss: {avg_train_loss:.2f}")
        
        print("Training complete!")
    
    def evaluate(self, test_reviews: list, test_sentiments: list) -> tuple:
        self.model.eval()
        with torch.no_grad():
            input_ids, attention_mask, sentiments = self.preprocess(test_reviews, test_sentiments)
            input_ids = input_ids.to(self.device)
            attention_mask = attention_mask.to(self.device)
            sentiments = sentiments.to(self.device)
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
        
        predictions = predictions.cpu()
        sentiments = sentiments.cpu()
        
        accuracy = metrics.accuracy_score(sentiments, predictions)
        precision = metrics.precision_score(sentiments, predictions)
        recall = metrics.recall_score(sentiments, predictions)
        f1_score = metrics.f1_score(sentiments, predictions)
        confusion_matrix = metrics.confusion_matrix(sentiments, predictions)
        
        # Plot ROC curve
        fpr, tpr, thresholds = metrics.roc_curve(sentiments, predictions)
        roc_auc = metrics.auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.legend(loc="lower right")
        plt.show()
        
        return accuracy, precision, recall, f1_score, confusion_matrix
    
    def save_model(self, filepath: str):
        if self.model:
            torch.save(self.model.state_dict(), filepath)
            print("Model saved!")
        else:
            print("No model to save.")


if __name__ == "__main__":
    classifier = SentimentClassifier()
    dataset_file = "/Users/camille/repo/Hetic/projet_gouv/scraping/data_nlp/google_reviews_RGPD.csv"
    classifier.train(dataset_file)
    
    dataset_test = pd.read_csv(dataset_file)
    dataset_test = dataset_test.sample(frac=0.2, random_state=200)
    test_reviews = dataset_test['review_text'].values.tolist()
    test_sentiments = dataset_test['sentiment'].values.tolist()
    
    accuracy, precision, recall, f1_score, confusion_matrix = classifier.evaluate(test_reviews, test_sentiments)
    classifier.save_model("./sentiments.pt")


/Users/camille/repo/Hetic/projet_gouv/scraping/clean_data_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkp

RuntimeError: MPS backend out of memory (MPS allocated: 11.10 GB, other allocations: 6.93 GB, max allowed: 18.13 GB). Tried to allocate 120.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [23]:
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'


In [ ]:
percentage_pos = result_pos_df['pos_info_predicted'].sum() / len(result_pos_df['pos_info_predicted'])*100
percentage_neg_ingo = result_pos_df['neg_info_predicted'].sum() / len(result_pos_df['neg_info_predicted'])*100